# Chapter 10 - Fine Tuning Network Hyperparameters

Finding the optimal arragement of hyperparaters is difficult and critical for model success. Examples of some parameters to tune are:
- Number of layers
- Neurons per layer
- Activation functions
- Weight initializations

One option is trying many combinations of parameters to see which work bestt on a validation set, such as using ```GridSearchCV``` or ```RandomizedSeachCV```. To use these, we need to wrap Keras models in objects that will mimic regualr Scikit-Learn regressors. The first step is to create a function that wil build and compile a Keras model, given a set of hyperparameters. 

In [7]:
import tensorflow as tf
from tensorflow import keras 
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full
)

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

Create a function to create a simple sequential model for univariate regression.

In [10]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

Create a ```KerasRegressor``` based on the ```build_model()``` function. The ```KerasRegressor``` object is a thin wrapper around the Keras model which is built by envoking the build_model function. It was created using the default arguments we provided. Now this can be used like a regular sci-kit learn regressor: train it using ```fit()``` and evaluate it using the ```score()``` method. 

In [11]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

<ipython-input-11-651c14c6d32f>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [15]:
keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 1s 1ms/step - loss: 1.1202 - val_loss: 0.6613
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.6741 - val_loss: 0.5619
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5841 - val_loss: 0.5424
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5487 - val_loss: 0.4980
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5197 - val_loss: 0.5187
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5061 - val_loss: 0.5026
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4925 - val_loss: 0.5121
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4843 - val_loss: 0.5254
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4777 - val_loss: 0.5478
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4717 - val_lo

In [17]:
mse_test = keras_reg.score(X_test, y_test)

162/162 [==============================] - 0s 911us/step - loss: 0.4305


Now lets find the best model by changing the number of hidden layers and neurons. Since there are many hyperparameters, it is better to use a randomized search rather than a GridSearch.

In [21]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np  

param_distribs = {
    'n_hidden' : [0,1,2,3],
    'n_neurons' : np.arange(1, 100),
    'learning_rate' : reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


c:\Users\ander\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 2.6039 - val_loss: 1.2351
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.9356 - val_loss: 0.8117
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7699 - val_loss: 0.7386
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7230 - val_loss: 0.7054
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6942 - val_loss: 0.6788
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6704 - val_loss: 0.6569
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6494 - val_loss: 0.6378
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6309 - val_loss: 0.6210
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6143 - val_loss: 0.6036
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5992 - val_loss: 0.5886
E

c:\Users\ander\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 2.3429 - val_loss: 1.1429
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.9273 - val_loss: 0.7690
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7598 - val_loss: 0.6886
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7141 - val_loss: 0.6471
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6853 - val_loss: 0.6187
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6616 - val_loss: 0.5932
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6423 - val_loss: 0.5740
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6249 - val_loss: 0.5580
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6098 - val_loss: 0.5437
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5965 - val_loss: 0.5319
E

...

c:\Users\ander\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 3ms/step - loss: 0.9855 - val_loss: 0.6661
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 0.6059 - val_loss: 0.5419
Epoch 3/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5423 - val_loss: 0.4998
Epoch 4/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5013 - val_loss: 0.4716
Epoch 5/100
242/242 [==============================] - 1s 3ms/step - loss: 0.4789 - val_loss: 0.4707
Epoch 6/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4665 - val_loss: 0.4603
Epoch 7/100
242/242 [==============================] - 1s 3ms/step - loss: 0.4599 - val_loss: 0.4756
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4513 - val_loss: 0.4762
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4447 - val_loss: 0.4727
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4378 - val_loss: 0.4917
E

c:\Users\ander\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 3ms/step - loss: 1.0686 - val_loss: 0.8175
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 0.9209 - val_loss: 0.7093
Epoch 3/100
242/242 [==============================] - 1s 3ms/step - loss: 1.1264 - val_loss: 0.5003
Epoch 4/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5304 - val_loss: 0.4689
Epoch 5/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4943 - val_loss: 0.4780
Epoch 6/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4762 - val_loss: 0.5269
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4669 - val_loss: 0.5282
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4545 - val_loss: 0.5670
Epoch 9/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4468 - val_loss: 0.6043
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4385 - val_loss: 0.6712
E

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x00000285D5D46400>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000285E02C0280>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

Search can last a very long time depending on the model complexity and hardware speed. To access the best model and its score:

```model.best_params_```

```model.best_score_```

In [22]:
rnd_search_cv.best_params_

{'learning_rate': 0.02330663662113993, 'n_hidden': 3, 'n_neurons': 38}